## Getting PostGameCarnageReports (PGCRs) from Destiny

### Step Zero

In order for this to work, you need to first request an API-Key from Bungie.net

I have included my own key commented out; however, I would prefer if you did not use that and instead got your own. 

You can do so here: http://destinyapiguide.azurewebsites.net/getKey

### First Step -- dependencies and imports

    1) Pandas (to make use of the DataFrame)
    
    2) requests (to call Bungie's API)
    
    3) deepcopy (from copy)
    
    4) MatchClass (a custom class to hold the data from each match we call)


In [6]:
import pandas as pd
import requests
from copy import deepcopy
import MatchClass as MC

### Second Step -- global variables

In [12]:
#Used in the combineTypeKills function -- enumerates all possible ways for a player to kill another player 
# another player and compresses that data into 4 features

weaponTypes = {}
weaponTypes["primary"] = ["weaponKillsAutoRifle", "weaponKillsScoutRifle", "weaponKillsHandCannon", "weaponKillsPulseRifle"]
weaponTypes["secondary"] = ["weaponKillsSniperRifle", "weaponKillsFusionRifle", "weaponKillsShotgun", "weaponKillsSidearm"]
weaponTypes["heavy"] = ["weaponKillsRocketLauncher", "weaponKillsMachineGun", "weaponKillsSwords"]
weaponTypes["other"] = ["weaponKillsGrenade", "weaponKillsSuper", "weaponKillsMelee"]

# The grouping of weapon types is Bungie's grouping where AutoRifles (automatic weapons), Scout Rifles (single fire weapons), 
# Hand Cannons (revolvers), and Pulse Rifles (burst fire weapons) are all grouped together in a slot called 'Primary'
# Likewise for the other 2 weapon types. 

# I chose to combine Grenades, Supers (one shot kills), and melee kills into one metric due to the ability-based and 
# miscellaneous nature of each of these killing methods.


Kernaldf = pd.read_csv("batch2.csv") #Reads in 50,000 incomplete Destiny Matche records

dflist = Kernaldf['instanceId'].tolist()    #We only care about the matchID so we save that to 
                                            #use as the seed to call complete matches
    
matches = [] #list to hold all of our matches
temp = None #used to hold data temporarily before being pushed to "matches"

### Third Step -- Utility functions for later

Up first comes a function which takes different granular data features and combines them into one larger feature

In [2]:
def combineTypeKills(PJR, i, gunType):

    used = []
    killcounter = 0

    all_keys = PJR['Response']['data']['entries'][i]['extended']['values'].keys()

    for guns in gunType:
        if guns in all_keys:
            used.append(guns)
    #print used

    for used_guns in used:
        killcounter += PJR['Response']['data']['entries'][i]['extended']['values'][used_guns]['basic']['value']

    #print counter
    return killcounter

Second is a function which takes all of the data we've saved in memory so far and saves it to disk as a CSV.

In [3]:
def saveToDisk(matches):
    # for sm in matches:
    #     print sm.display()

    df = pd.DataFrame.from_dict(matches[0].getMatch(), orient='index')

    for i in range(1, len(matches)):
        df = df.append(pd.DataFrame.from_dict(matches[i].getMatch(), orient='index'))

    #print df.head(), df.shape

    df.to_csv('batch6Udacity.csv')
    print(df.shape)
    del matches[:]

### Step Four -- Pinging Bungie's API

Insert Description of the code below and how to use it in conjuction with the matchIDs.

In [11]:
for j in range(15000):
    if j%5000 == 0:
        print("On match ", j)

    match = str(dflist[j])

    PGCR_url = "http://www.bungie.net/Platform/Destiny/Stats/PostGameCarnageReport/" + match + "/"

    HEADERS = {"X-API-Key": "INSERT-API-KEY-HERE")

    #print("\n\n\nConnecting to Bungie: " + PGCR_url + "\n")
    #print("Fetching data for: PGCR!")
    res = requests.get(PGCR_url, headers=HEADERS)

    totalplayerCount = int(res.json()['Response']['data']['entries'][0]['values']['playerCount']['basic']['value'])

    for i in range(totalplayerCount):
        singleMatch = MC.SingleMatch(match)

        personId = res.json()['Response']['data']['entries'][i]['player']['destinyUserInfo']['membershipId']

        if "team" not in res.json()['Response']['data']['entries'][i]['values'].keys() \
                or "extended" not in res.json()['Response']['data']['entries'][i].keys()\
                or "values" not in res.json()['Response']['data']['entries'][i].keys():
            continue
        else:
            team = res.json()['Response']['data']['entries'][i]['values']['team']['basic']['displayValue']

        if team == '-':
            continue

        singleMatch.addPlayer(personId)
        singleMatch.addTeam(personId, team)
        singleMatch.addMatchIDtoPlayers(personId)

        singleMatch.addScore(personId, res.json()['Response']['data']['entries'][i]['score']['basic']['value'])
        singleMatch.addAVGScorePerLife(personId, res.json()['Response']['data']['entries'][i]['values']['averageScorePerLife']['basic']['value'])
        singleMatch.addKills(personId, res.json()['Response']['data']['entries'][i]['values']['kills']['basic']['value'])
        singleMatch.addAssists(personId, res.json()['Response']['data']['entries'][i]['values']['assists']['basic']['value'])
        singleMatch.addDeaths(personId, res.json()['Response']['data']['entries'][i]['values']['deaths']['basic']['value'])
        singleMatch.addStanding(personId, res.json()['Response']['data']['entries'][i]['values']['standing']['basic']['value'])
        singleMatch.addKD(personId,res.json()['Response']['data']['entries'][i]['values']['killsDeathsRatio']['basic']['value'])
        singleMatch.addKAD(personId, res.json()['Response']['data']['entries'][i]['values']['killsDeathsAssists']['basic']['value'])

        if ("averageKillDistance" not in res.json()['Response']['data']['entries'][i]['extended']['values'].keys()):
            None
        else:
            singleMatch.addAVG_K_DIST(personId, res.json()['Response']['data']['entries'][i]['extended']['values']['averageKillDistance']['basic']['value'])

        if ("averageScorePerKill" not in res.json()['Response']['data']['entries'][i]['extended']['values'].keys()):
            None
        else:
            singleMatch.addAVGScorePerKill(personId, res.json()['Response']['data']['entries'][i]['extended']['values']['averageScorePerKill']['basic']['value'])

        if ("combatRating" not in res.json()['Response']['data']['entries'][i]['extended']['values'].keys()):
            None
        else:
            singleMatch.addCR(personId, int(res.json()['Response']['data']['entries'][i]['extended']['values']['combatRating']['basic']['value']))

        singleMatch.addPrimaries(personId, combineTypeKills(res.json(), i, weaponTypes["primary"]))
        singleMatch.addSecondaries(personId, combineTypeKills(res.json(), i, weaponTypes["secondary"]))
        singleMatch.addHeavy(personId, combineTypeKills(res.json(), i, weaponTypes["heavy"]))
        singleMatch.addAbilityKills(personId, combineTypeKills(res.json(), i, weaponTypes["other"]))

        singleMatch.addActiveDuration(personId, int(res.json()['Response']['data']['entries'][i]['values']['activityDurationSeconds']['basic']['value']))
        singleMatch.addCompletion(personId, res.json()['Response']['data']['entries'][i]['values']['completionReason']['basic']['value'])

        if len(res.json()['Response']['data']['teams']) == 2:

            if res.json()['Response']['data']['entries'][i]['values']['team']['basic']['displayValue'] == "Alpha":

                singleMatch.addpercentContrib(personId,
                                              singleMatch.getScore(personId),
                                              res.json()['Response']['data']['teams'][0]['score']['basic']['value'])
            else:
                singleMatch.addpercentContrib(personId,
                                              singleMatch.getScore(personId),
                                              res.json()['Response']['data']['teams'][1]['score']['basic']['value'])

        temp = deepcopy(singleMatch)
        matches.append(temp)

        # if count%5 == 0:
        #     temp.display()
               
               
               
saveToDisk(matches)

SyntaxError: invalid syntax (<ipython-input-11-131fd75c5beb>, line 9)